In [2]:
#CS19MDS11017
#Shravya Ch
#recommendation systems assignment exercise 5

In [1]:
#importing packages
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
#reading the data and items as columns for dataframe
data = pd.read_csv('C://Users//home//Downloads//dataset//user-shows.txt',header = None,sep=' ')
cols = pd.read_csv('C://Users//home//Downloads//dataset//shows.txt',header = None)
data.columns = list(cols[0])
data.head()

,The Situation Room with Wolf Blitzer,NBC Nightly News,Dancing with the Stars,The Colbert Report,Larry King Live,Everybody Loves Raymond,NHL Hockey,"Campbell Brown: No Bias, No Bull",NBA Basketball,Two and a Half Men,...,Dragon Tales,FOX News Sunday with Chris Wallace,Boston Legal,The Wedding Date,Intervention,The Unusuals,The People's Court,Never Back Down,Da Vinci's Inquest,Cardinals Live
0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
data.shape

(9985, 563)

In [115]:
#R and R transpose
r = data.to_numpy()
r_t = np.transpose(r)
print(r.shape,r_t.shape)

(9985, 563) (563, 9985)


In [116]:
#P and Q are calculated
user_num = np.matmul(r,r_t)
item_num = np.matmul(r_t,r)
p =np.diag(np.diag(user_num))
q =np.diag(np.diag(item_num))

In [117]:
#calculating P and Q
p_modif = np.diag(np.apply_along_axis(lambda x: 1/np.sqrt(x),0,np.diag(user_num)))
q_modif = np.diag(np.apply_along_axis(lambda x: 1/np.sqrt(x),0,np.diag(item_num)))

In [118]:
#Similarity matrices for user and item are calculated as given 
user_sim = np.matmul(p_modif,np.matmul(user_num,p_modif))
item_sim = np.matmul(q_modif,np.matmul(item_num,q_modif))

In [119]:
#user-user and item-item collaborative filtering matrices are calculated
t_user = np.matmul(user_sim,r)
t_item = np.matmul(r,item_sim)

In [21]:
#recommended top 10 shows for user 20 using user-user collaborative filtering
user_user= np.flip(np.argsort(t_user[19])[-10:])
user_user,data.columns[user_user].tolist()

(array([ 96,  74, 140,  45,  60, 156,  68,  35, 137, 326]),
 ['FOX 28 News at 10pm',
  'Family Guy',
  '10TV News HD at 11pm',
  '2009 NCAA Basketball Tournament',
  'NBC 4 at Eleven',
  'Seinfeld',
  'Today',
  'SpongeBob SquarePants',
  '10TV Eyewitness News at 5:00',
  '10TV Eyewitness News at 6:00'])

In [22]:
#recommended top 10 shows for user 20 using item-item collaborative filtering
item_item = np.flip(np.argsort(t_item[19])[-10:])
item_item,data.columns[item_item].tolist()

(array([144,  96,  35,  74, 155, 173, 205,  63, 140, 145]),
 ['Dora, the Explorer',
  'FOX 28 News at 10pm',
  'SpongeBob SquarePants',
  'Family Guy',
  'Ni Hao, Kai-Lan',
  'Wonder Pets!',
  'The Backyardigans',
  'Wow Wow Wubbzy',
  '10TV News HD at 11pm',
  'Go, Diego, Go!'])

In [12]:
from scipy.stats import kendalltau

In [23]:
#kendall rank coefficient for user-user and item-item
kendalltau(user_user,item_item)

KendalltauResult(correlation=0.19999999999999998, pvalue=0.4843127204585538)

In [ ]:
#saving the file in format for MyMediaLite  
pd.DataFrame(data=np.array(ind).transpose()).to_csv("C://Users//home//Downloads//train.csv")

In [2]:
#reading the prediction file ,predictions from MyMediaLite
rank = pd.read_csv("C://Users//home//Downloads//MyMediaLite-3.11//lib//mymedialite//predicth.txt",sep='\t',header=None)

In [3]:
rank.head()

,0,1
0,0,"[520:0.2594911,471:0.2493659,479:0.2445541,151..."
1,1,"[174:0.1820814,191:0.1699065,189:0.1687232,118..."
2,2,"[209:0.3720511,256:0.3525813,300:0.3466443,148..."
3,3,"[405:0.1483086,286:0.1478028,560:0.1445312,254..."
4,4,"[47:0.1582297,23:0.1521604,7:0.150713,134:0.15..."


In [4]:
#processing the data for user 20 
x= rank[1].iloc[19].split(',')
result =[]
for i,p in zip(range(0,len(x)),x):
    if i==0:
        result.append(int(p.split('[')[1].split(':')[0]))
    elif i==(len(x)-1):
        result.append(int(p.split(']')[0].split(':')[0]))
    else:
        result.append(int(p.split(":")[0]))

In [7]:
#recommendeed top 10 shows for user 20 using ItemKNN recommender
np.array(result)[0:10],data.columns[np.array(result)[0:10]].tolist()

(array([234,  48,  37, 543, 490, 477, 280, 553, 489, 222]),
 ["Jack's Big Music Show",
  'Max and Ruby',
  'Curious George',
  'Sesame Street',
  'Clifford the Big Red Dog',
  'The NewsHour with Jim Lehrer',
  'Olivia',
  'Dragon Tales',
  'Caillou',
  'Little Einsteins'])

In [13]:
#kendall rank coefficient for (item-item,ItemKNN) and (user-user,Item-KNN)
print(kendalltau(item_item,np.array(result)[0:10]),
      kendalltau(user_user,np.array(result)[0:10]))

KendalltauResult(correlation=-0.022222222222222223, pvalue=1.0) KendalltauResult(correlation=-0.5555555555555555, pvalue=0.02860945767195767)


In [14]:
#reading predictions from WRMF recommender
rank_wrmf = pd.read_csv("C://Users//home//Downloads//MyMediaLite-3.11//lib//mymedialite//predict_wrmf.txt",sep='\t',header=None)

In [15]:
rank_wrmf.head()

,0,1
0,0,"[117:0.7038108,45:0.6045014,46:0.5848253,82:0...."
1,1,"[11:0.4805392,384:0.4490533,177:0.4112435,8:0...."
2,2,"[125:0.6775061,148:0.6553589,156:0.6426778,1:0..."
3,3,"[155:0.4083053,173:0.3879905,48:0.3827862,145:..."
4,4,"[11:0.4503127,384:0.4427823,57:0.4091523,8:0.4..."


In [16]:
x= rank_wrmf[1].iloc[19].split(',')
result_wrmf =[]
for i,p in zip(range(0,len(x)),x):
    if i==0:
        result_wrmf.append(int(p.split('[')[1].split(':')[0]))
    elif i==(len(x)-1):
        result_wrmf.append(int(p.split(']')[0].split(':')[0]))
    else:
        result_wrmf.append(int(p.split(":")[0]))

In [19]:
#recommendeed top 10 shows for user 20 using WRMF recommender
np.array(result_wrmf)[0:10],data.columns[np.array(result_wrmf)[0:10]].tolist()

(array([ 48, 280,  77, 192, 208, 195, 207, 489, 222, 234]),
 ['Max and Ruby',
  'Olivia',
  "Disney's Mickey Mouse Clubhouse",
  'Handy Manny',
  'Tasty Time with ZeFronk',
  'Imagination Movers',
  'Happy Monster Band',
  'Caillou',
  'Little Einsteins',
  "Jack's Big Music Show"])

In [24]:
#kendall rank coefficiet for (item-item,WRMF),(user-user,WRMF) and (ItemKNN,WRMF)
print(kendalltau(item_item,np.array(result_wrmf)[0:10]),
      kendalltau(user_user,np.array(result_wrmf)[0:10]),
      kendalltau(np.array(result)[0:10],np.array(result_wrmf)[0:10]))

KendalltauResult(correlation=-0.1111111111111111, pvalue=0.7274895282186948) KendalltauResult(correlation=-0.1111111111111111, pvalue=0.7274895282186948) KendalltauResult(correlation=0.1111111111111111, pvalue=0.7274895282186948)
